In [ ]:
from itertools import starmap
from functools import partial
from collections import namedtuple
import re
import datetime

from IPython import display
import traitlets
from ipywidgets import Button, Text, SelectMultiple, HBox, VBox, Textarea, Image, Label, HTML, Dropdown, IntProgress
import pandas
import jinja2
from pony import orm

In [ ]:
import db
db.use_db('course.sqlite')

In [ ]:
### Javascript kludges to add autocompletion to text boxes and enable multiple selections

autocomplete = jinja2.Template("""
  $( function() {
    var availableTags = [
    {%- for option in options %}
        "{{ option }}",
    {%- endfor %}
    ];
    $( ":input[{{ selector }}]" ).autocomplete({
      minLength: 3,
      source: availableTags
    });
  } );
""")

# Widget definitions

In [ ]:
### Views -- all the individual widgets and UI state

class AppState(traitlets.HasTraits):
    # Immutable
    students = traitlets.List(traitlets.Unicode(), read_only=True)
    exams = traitlets.List(traitlets.Unicode(), read_only=True)

    exam_id = traitlets.Integer()

    submission_id = traitlets.Integer()
    student = traitlets.Unicode()
    email = traitlets.Unicode()

    # --- Defaults and validation ---
    ## students
    @traitlets.default('students')
    def _default_students(self):
        with orm.db_session:
            return tuple(starmap('{}, {}, {}'.format,
                                 orm.select((s.id, s.first_name, s.last_name)
                                            for s in db.Student)))
    
    ## exams
    @traitlets.default('exams')
    def _default_exams(self):
        with orm.db_session:
            exams = orm.select(e for e in db.Exam).order_by(lambda e: e.id)
            return list(e.name for e in exams)

    ## exam_id
    @traitlets.default('exam_id')
    def _default_exam_id(self):
        with orm.db_session:
            return db.Exam.select().first().id

    @traitlets.validate('exam_id')
    def _valid_exam(self, proposal):
        with orm.db_session():
            if db.Exam.get(id=proposal['value']) is None:
                raise traitlets.TraitError('Unknown exam id.')
        return proposal['value']

    ## submission_id
    @traitlets.default('submission_id')
    def _default_submission_id(self):
        with orm.db_session:
            sid = (orm.select(s.id for s in db.Submission
                             if s.exam.id == self.exam_id)
                      .order_by(lambda x:x)
                      .first())
            return sid

    @traitlets.validate('submission_id')
    def _valid_submission_id(self, proposal):
        with orm.db_session():
            sub = db.Submission.get(id=proposal['value'])
            if sub is None:
                raise traitlets.TraitError('Unknown submission id.')
            if sub.exam.id != self.exam_id:
                raise traitlets.TraitError('Submission from a different exam.')
        return proposal['value']
    
    @traitlets.default('student')
    def _default_student(self, sid=None):
        sid = sid or self.submission_id
        with orm.db_session:
            stud = db.Submission[sid].student
            if stud is None:
                return ''

            return '{}, {}, {}'.format(stud.id, stud.first_name, stud.last_name)
    
    @traitlets.default('email')
    def _default_email(self):
        match = re.match(r'(\d{7}), (.+), (.+)', self.student)
        if match is None:
            return ''
        else:
            number, *_ = match.groups()

        with orm.db_session:
            stud = db.Student.get(id=int(number))
            if stud is not None:
                return stud.email or ''
            return ''

    ## --- Trait relations ---
    @traitlets.observe('exam_id')
    def _change_exam(self, change):
        self.commit_student()
        self.submission_id = self._default_submission_id()

    @traitlets.observe('submission_id')
    def commit_student(self, change=None):
        if not self.student:
            return
        if change is None:
            sid = self.submission_id
        else:
            sid = change['old']
        # Parse the student entry
        student_nr, first_name, last_name = re.match(r'(\d{7}), (.+), (.+)',
                                                     self.student).groups()
        first_name = first_name.strip()
        last_name = last_name.strip()
        email = self.email.strip()
        with orm.db_session:
            stud = db.Student.get(id=int(student_nr)) or\
                   db.Student(id=student_nr, first_name=first_name, last_name=last_name)
            if stud.email is None:
                stud.email = email

            db.Submission[sid].student = stud

        self.student = self._default_student(change and change['new'])
        
    @traitlets.observe('student')
    def _update_email(self, change):
        self.email = self._default_email()

    ## --- Navigation ---
    def exam_to_id(self, exam_name):
        with orm.db_session:
            exam = db.Exam.get(name=exam_name)
            if not exam:
                raise ValueError('No exam {}'.format(exam_name))
            return exam.id
    
    def next_submission(self, *_):
        with orm.db_session:
            subs = orm.select(s.id for s in db.Submission
                              if s.exam.id == self.exam_id
                                 and s.id > self.submission_id)
            result = subs.order_by(lambda x: x).first()
            if result is None:
                self.submission_id = self._default_submission_id()
            else:
                self.submission_id = result

    def previous_submission(self, *_):
        with orm.db_session:
            subs = orm.select(s.id for s in db.Submission
                              if s.exam.id == self.exam_id
                                 and s.id < self.submission_id)

            result = subs.order_by(lambda x: -x).first()
            if result is None:
                subs = orm.select(s.id for s in db.Submission
                                  if s.exam.id == self.exam_id)
                result = subs.order_by(lambda x: -x).first()

            self.submission_id = result

    def signature_image(self, sub_id):
        with orm.db_session:
            path = db.Submission[sub_id].signature_image_path
        with open(path, 'rb') as f:
            image = f.read()
        
        return image
    
    def submissions_number(self, exam_id=None):
        with orm.db_session:
            return orm.count(db.Submission.select(lambda s: s.exam.id == (exam_id and self.exam_id)))

    def progress(self, sid=None):
        with orm.db_session:
            return orm.count(db.Submission.select(lambda s: s.exam.id == self.exam_id
                                                            and s.id < (sid and self.submission_id)))


state = AppState()

def is_not_student(student):
    try:
        student_nr, first_name, last_name = re.match(r'(\d{7}), (.+), (.+)',
                                                     student).groups()
        return False
    except AttributeError:
        return True


exam = Dropdown(options=state.exams, value=db.Exam[state.exam_id].name)
traitlets.directional_link((exam, 'value'), (state, 'exam_id'), state.exam_to_id)
traitlets.directional_link((state, 'student'), (exam, 'disabled'), is_not_student)


prev_nr = Button(icon='fa-step-backward', description=' previous')
prev_nr.on_click(state.previous_submission)
traitlets.directional_link((state, 'student'), (prev_nr, 'disabled'), is_not_student)

next_nr = Button(icon='fa-step-forward', description=' next')
next_nr.on_click(state.next_submission)
traitlets.directional_link((state, 'student'), (next_nr, 'disabled'), is_not_student)

student_info = Text(placeholder='number, first, last')
traitlets.link((state, 'student'), (student_info, 'value'))

email_info = Text(placeholder='email')
traitlets.link((state, 'email'), (email_info, 'value'))

signature = Image(format='png', width='100%')
traitlets.directional_link((state, 'submission_id'), (signature, 'value'), state.signature_image)

number_progress = IntProgress(
    min=0,
    step=1,
    description='Progress:',
    bar_style='success',
    orientation='horizontal',
)
traitlets.directional_link((state, 'exam_id'), (number_progress, 'max'), state.submissions_number)
traitlets.directional_link((state, 'submission_id'), (number_progress, 'value'), state.progress);

# Layout

In [ ]:
HBox([exam, prev_nr, student_info, email_info, next_nr])

In [ ]:
signature

In [ ]:
number_progress

In [ ]:
# apply javascript kludges
# must trigger after the widgets were rendered

selector = "placeholder='{}'".format(student_info.placeholder)
display.display_javascript(display.Javascript(autocomplete.render(selector=selector, options=state.students)))